In [1]:
import json
import requests
import praw
import numpy
import time

reddit = praw.Reddit(client_id='tiBit4kBeuoOTA', 
                     client_secret='7-HPDa9q92MS0a-AnUVYb72QDzU', 
                     user_agent='TorkilSamStat', 
                     username='TorkSamStat', 
                     password='bbybbyduerinnafor')

# STATIC VARIABLES
base_url = 'https://api.pushshift.io/reddit/search/comment/'
wanted_count_auth = 3
wanted_count_sub = 2
unwanted_users = ["[deleted]", "AutoModerator"]
unwanted_count = len(unwanted_users)
request_limit = 5
request_count = 0
request_sum = 0
sleep_time = 10
    
# FUNCTIONS
def getKeysFromDict(data):
    users = []
    for obj in data:
        if obj['key'] not in unwanted_users:
            users.append(obj['key'])
    return users

def getMostActiveAuthorsInSubReddit(subreddit):
    global request_count
    global request_sum

    if request_count >= request_limit:
        print("Sleeping...")
        time.sleep(sleep_time)
        request_count = 0
        print("Reseting request_count to 0")
        print("Resuming...")
        
    url = base_url + '?since=100d&subreddit={}&aggs=author&agg_size={}'.format(subreddit, wanted_count_auth + unwanted_count)
    response_raw = requests.get(url)
    request_count += 1
    request_sum += 1
    

    print("Requests performed:\t\t\t", request_count)
    print("Total requests performed:\t\t", request_sum)
    print("Percent completed:\t\t\t", 555)

    print(url, response_raw)
    response = response_raw.json()
    #print(response)
    fetched_authors = getKeysFromDict(response['aggs']['author'])
    print("*******************")
    print()

    return fetched_authors


def getMostActiveSubRedditForAuthor(author):
    global request_count
    global request_sum
    
    if request_count >= request_limit:
        print("Sleeping...")
        time.sleep(sleep_time)
        request_count = 0
        print("Reseting request_count to 0")
        print("Resuming...")
        
    url = base_url + '?since=100d&author={}&aggs=subreddit&agg_size={}'.format(author, wanted_count_sub)
    response_raw = requests.get(url)
    request_count += 1
    request_sum += 1

    print("Requests performed:\t\t\t", request_count)
    print("Total requests performed:\t\t", request_sum)
    print("Percent completed:\t\t\t", 555)    

    print(url, response_raw)
    response = response_raw.json()
    #print(response)
    fetched_subreddits = getKeysFromDict(response['aggs']['subreddit'])
    print("Detected subreddits:\t\t\t", fetched_subreddits)
    for sub in used_subreddits:
            if (sub not in used_subreddits):
                used_subreddits.append(sub)
    print("*******************")
    print()

    
    return fetched_subreddits

In [3]:
%%time

# Global variables
used_subreddits = []
used_authors = []
start = time.time()


def get_relations(relational_array, subreddit, wanted_subcount):
    global request_count
    global request_sum
    time_elapsed = time.time() - start
    print("Time elapsed: \t\t\t\t", round(time_elapsed,2), "seconds")
    
    if wanted_subcount == 0:
        return []
    
    # Add subreddit to used_subreddits 
    used_subreddits.append(subreddit)
    
    # Retrieve users from subreddit and add to request_count
    users = getMostActiveAuthorsInSubReddit(subreddit)

    # Retrieve subs from active users in subreddit
    for user in users:
            subs = getMostActiveSubRedditForAuthor(user)
            used_authors.append(user)

            # Enter new level
            for sub in subs:
                if sub != subreddit:
                    relational_array.append([subreddit, sub])

                if sub not in used_subreddits:
                    relational_array.extend(get_relations([], sub, wanted_subcount - 1))
    
    return relational_array


rels = get_relations([], 'The_Donald', 2)
rels

Time elapsed: 				 0.0 seconds
Requests performed:			 4
Total requests performed:		 4
Percent completed:			 555
https://api.pushshift.io/reddit/search/comment/?since=100d&subreddit=The_Donald&aggs=author&agg_size=5 <Response [200]>
*******************

Requests performed:			 5
Total requests performed:		 5
Percent completed:			 555
https://api.pushshift.io/reddit/search/comment/?since=100d&author=MAGA_MAGGING_MAGA&aggs=subreddit&agg_size=2 <Response [200]>
Detected subreddits:			 ['The_Donald', 'AmItheAsshole']
*******************

Time elapsed: 				 7.04 seconds
Sleeping...
Reseting request_count to 0
Resuming...
Requests performed:			 1
Total requests performed:		 6
Percent completed:			 555
https://api.pushshift.io/reddit/search/comment/?since=100d&subreddit=AmItheAsshole&aggs=author&agg_size=5 <Response [200]>
*******************

Requests performed:			 2
Total requests performed:		 7
Percent completed:			 555
https://api.pushshift.io/reddit/search/comment/?since=100d&author=tenaci

[['The_Donald', 'AmItheAsshole'],
 ['AmItheAsshole', 'SampleSize'],
 ['AmItheAsshole', 'Portland'],
 ['AmItheAsshole', 'ModSupport'],
 ['The_Donald', 'trump'],
 ['trump', 'The_Donald'],
 ['trump', 'The_Donald'],
 ['trump', 'AskTrumpSupporters'],
 ['The_Donald', 'florida'],
 ['florida', 'Unemployment'],
 ['florida', 'dataisbeautiful'],
 ['florida', 'PurplePillDebate'],
 ['florida', 'Unemployment']]

### 